In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sys import argv
%matplotlib

Using matplotlib backend: <object object at 0x7f6418796f90>


In [2]:
def f_up(w, lam, alpha, mu):
    return lam*(w)**mu

def f_down(w, lam, alpha, mu):
    return -lam*alpha*w

In [3]:
#G_h = 20  # Glucose
G_h = .1  # ATP
def lam_e(A):
    #x = (A - G_h)  # Glucose
    #lam_n = lam + .4/(1 + np.exp(-x))  # Glucose
    intens = 2
    x = intens*(A - G_h)/G_h # ATP
    lm = .3
    lam_n = lam + (lm/(1 + np.exp(-x)) -lm/2)  # APT
    lam_n = lam_n*(lam_n >= 0)
    return lam_n

def mu_e(A):
    #x = (A - G_h)  # Glucose
    #mu_n = mu + .2/(1 + np.exp(-x))  # Glucose
    intens = 3
    x = intens*(A - G_h)/G_h # ATP
    m = .2
    mu_n = mu + (m/(1 + np.exp(-x)) - m/2)  # ATP
    mu_n = mu_n*(mu_n >= 0)
    return mu_n

def alpha_e(A):
    #x = (A - G_h)  # Glucose
    #alpha_n = alpha + .2/(1 + np.exp(-x))  #  Glucose
    intens = 2.2 
    x = intens*(A - G_h)/G_h # ATP
    a = -.2
    alpha_n = alpha + (a/(1 + np.exp(-x)) - a/2)  # ATP
    alpha_n = alpha_n*(alpha_n >= 0)
    return alpha_n

def fe_up(w, lam, alpha, mu, A, dep):
    if dep == 'lam' or 'lam_up&alpha_down':
        return lam_e(A)*(w)**(mu)
    elif dep == 'mu':
        return lam*(w)**(mu_e(A))
    elif dep == 'mu&alpha':
        return lam*(w)**(mu_e(A))

def fe_down(w, lam, alpha, mu, A, dep):
    if dep == 'lam':
        return -lam_e(A)*alpha*w
    if dep == 'lam_up&alpha_down':
        return -lam*alpha_e(A)*w
    elif dep == 'mu':
        return -lam*alpha*w
    elif dep == 'mu&alpha':
        return -lam*alpha_e(A)*w

In [4]:
def delta_w(w, delta_t, lam, alpha, mu):
    if delta_t > 0:
        Dw = f_up(w, lam, alpha, mu)*np.exp(-abs(delta_t)/tau_up) 
    elif delta_t <= 0:
        Dw = f_down(w, lam, alpha, mu)*np.exp(-abs(delta_t)/tau_down) 
    return np.real(Dw)

In [5]:
def delta_e_w(w, delta_t, lam, alpha, mu, A, dep):
    if delta_t > 0:
        Dw = fe_up(w, lam, alpha, mu, A, dep)*np.exp(-abs(delta_t)/tau_up) 
    elif delta_t <= 0:
        Dw = fe_down(w, lam, alpha, mu, A, dep)*np.exp(-abs(delta_t)/tau_down) 
    return np.real(Dw)

In [6]:
## Standar STDP Rules parameters
tau_up = 20
tau_down = 20
lam = 0.1
alpha = 0.11
mu = 0.4

In [7]:
def stimuli_60s(w_ini, spikes_timing):
    grad_w = []
    grad_w_perc = []
    for d_time in spikes_timing:
        grad_w_temp = []
        w = w_ini
        grad_w_temp.append(w)
        if abs(d_time) < 2:
            w = np.nan
            grad_w_temp.append(w)
        for _ in range(0, 61):
            grad_w_temp.append(delta_w(w, d_time, lam, alpha, mu))
            w+=grad_w_temp[-1]
        grad_w.append(w)
        grad_w_perc.append((w- w_ini)/w_ini*100)
    return grad_w_perc

In [8]:
spikes_timing = np.linspace(-100,100,200)
w_ini = 17
grad_w17_perc = stimuli_60s(w_ini, spikes_timing)
w_ini = 50
grad_w50_perc = stimuli_60s(w_ini, spikes_timing)
w_ini = 100
grad_w100_perc = stimuli_60s(w_ini, spikes_timing)

In [9]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.axvspan(-20, 20, color = "0.9")
ax.set_xlabel('time (ms)', fontsize=18)
ax.set_ylabel('PSC change [%]', fontsize=18)
plt.title('STDP: 60 seg 1 Hz stimulation', fontsize=20)
ax.plot(spikes_timing, grad_w17_perc, c = 'b', linewidth=3.0, label = 'W init = 17 pA')
ax.plot(spikes_timing, grad_w50_perc, c = 'r', linewidth=3.0, label = 'W init = 50 pA')
ax.plot(spikes_timing, grad_w100_perc, c = 'g', linewidth=3.0, label = 'W init = 100 pA')
ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)
ax.set_ylim(-55,105)
ax.legend()
ax.grid()
plt.show()

In [ ]:
## ATP dependent STDP Rules parameters
tau_up = 20
tau_down = 20
lam = 0.1
alpha = .11
mu = 0.4

In [ ]:
def stimuli_60s_e(w_ini, spikes_timing, A, dep):
    grad_w = []
    grad_w_perc = []
    for d_time in spikes_timing:
        grad_w_temp = []
        w = w_ini
        grad_w_temp.append(w)
        if abs(d_time) < 2:
            w = np.nan
            grad_w_temp.append(w)
        for _ in range(0, 61):
            grad_w_temp.append(delta_e_w(w, d_time, lam, alpha, mu, A, dep))
            w+=grad_w_temp[-1]
        grad_w.append(w)
        grad_w_perc.append((w - w_ini)/w_ini*100)
    return grad_w_perc

In [ ]:
spikes_timing = np.linspace(-100,100,200)
w_ini = 50
A = 8
grad_w50_A8_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'lam')
w_ini = 50
A = 20
grad_w50_A20_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'lam')
w_ini = 50
A = 30
grad_w50_A30_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'lam')

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.axvspan(-20, 20, color = "0.9")
ax.set_xlabel('time (ms)', fontsize=18)
ax.set_ylabel('PSC change [%]', fontsize=18)
plt.title('STDP: 60 seg 1 Hz stimulation. dependence in $\lambda : = \lambda(A)$', fontsize=20)
ax.plot(spikes_timing, grad_w50_A8_perc, c = 'r', linestyle = '--' , linewidth=3.0, label = 'A = 8 mM')
ax.plot(spikes_timing, grad_w50_A20_perc, c = 'r', linestyle = '-.', linewidth=3.0, label = 'A = 20 mM')
ax.plot(spikes_timing, grad_w50_A30_perc, c = 'r', linestyle = ':', linewidth=3.0, label = 'A = 30 mM')
ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)
ax.legend()
ax.grid()
plt.show()

In [ ]:
spikes_timing = np.linspace(-100,100,200)
w_ini = 50
A = 8
grad_w50_A8_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'mu')
w_ini = 50
A = 20
grad_w50_A20_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'mu')
w_ini = 50
A = 30
grad_w50_A30_perc = stimuli_60s_e(w_ini, spikes_timing, A, 'mu')

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.axvspan(-20, 20, color = "0.9")
ax.set_xlabel('time (ms)', fontsize=18)
ax.set_ylabel('PSC change [%]', fontsize=18)
plt.title(r'STDP: 60 seg 1 Hz stimulation. dependence in $\mu:=\mu(A)$', fontsize=20)
ax.plot(spikes_timing, grad_w50_A8_perc, c = 'r', linestyle = '--' , linewidth=3.0, label = 'A = 8 mM')
ax.plot(spikes_timing, grad_w50_A20_perc, c = 'r', linestyle = '-.', linewidth=3.0, label = 'A = 20 mM')
ax.plot(spikes_timing, grad_w50_A30_perc, c = 'r', linestyle = ':', linewidth=3.0, label = 'A = 30 mM')
ax.tick_params(axis="x", labelsize=20)
ax.tick_params(axis="y", labelsize=20)
ax.legend()
ax.grid()
plt.show()

In [ ]:
def stimuli_60s_sngl(w_ini, spikes_timing, A):
    grad_w = []
    grad_w_perc = []
    for d_time in spikes_timing:
        grad_w_temp = []
        w = w_ini
        grad_w_temp.append(w)
        if abs(d_time) < 2:
            w = np.nan
            grad_w_temp.append(w)
        for _ in range(0, 61):
            grad_w_temp.append(delta_w(w, d_time, lam, alpha, mu))
            w+=grad_w_temp[-1]
        grad_w.append(w)
        grad_w_perc.append((w - w_ini)/w_ini*100)
    return grad_w_perc

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('time (ms)', fontsize=12)
#ax.set_ylabel('glucose [mM]', fontsize=18)
ax.set_ylabel('[ATP] (mM)', fontsize=12)
ax.set_zlabel('PSC change (%)', fontsize=12)

x = np.linspace(-100,100,200)
#y = np.linspace(8,50,30)  # Glucose
y = np.linspace(0.001,.1,30)  # ATP
X, Y = np.meshgrid(x, y)
zs = []
for A in y:
    zs.append(stimuli_60s_sngl(50, x, A))
zs = np.array(zs)
Z = zs.reshape(X.shape)

surf = ax.plot_surface(X, Y, Z, rstride=5, cstride=5, cmap='cool', alpha=.6,
                        linewidth=0, antialiased=True, vmin=np.nanmin(Z), vmax=np.nanmax(Z))

ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)
plt.title(r'STDP: 60 seg 1 Hz stimulation. Classic', fontsize=20)
ax.tick_params(axis="x", labelsize=10)
ax.tick_params(axis="y", labelsize=10)
ax.tick_params(axis="z", labelsize=10)

plt.show()

In [ ]:
def stimuli_60s_e_sngl(w_ini, spikes_timing, A, dep):
    grad_w = []
    grad_w_perc = []
    for d_time in spikes_timing:
        grad_w_temp = []
        w = w_ini
        grad_w_temp.append(w)
        if abs(d_time) < 2:
            w = np.nan
            grad_w_temp.append(w)
        for _ in range(0, 61):
            grad_w_temp.append(delta_e_w(w, d_time, lam, alpha, mu, A, dep))
            w+=grad_w_temp[-1]
        grad_w.append(w)
        grad_w_perc.append((w - w_ini)/w_ini*100)
    return grad_w_perc

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('time (ms)', fontsize=12)
#ax.set_ylabel('glucose [mM]', fontsize=18)
ax.set_ylabel('[ATP] (mM)', fontsize=12)
ax.set_zlabel('PSC change (%)', fontsize=12)

x = np.linspace(-100,100,200)
#y = np.linspace(8,50,30)  # Glucose
y = np.linspace(0.001,.1,30)  # ATP
X, Y = np.meshgrid(x, y)
zs = []
for A in y:
    zs.append(stimuli_60s_e_sngl(50, x, A, 'lam'))  # Only lam
    #zs.append(stimuli_60s_e_sngl(50, x, A, 'lam_up&alpha_down'))  # lam and alpha
zs = np.array(zs)
Z = zs.reshape(X.shape)

surf = ax.plot_surface(X, Y, Z, rstride=5, cstride=5, cmap='cool', alpha=.6,
                        linewidth=0, antialiased=True, vmin=np.nanmin(Z), vmax=np.nanmax(Z))

ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)
plt.title(r'STDP: 60 seg 1 Hz stimulation. dependence in $\lambda:=\lambda(A)$', fontsize=20)
ax.tick_params(axis="x", labelsize=10)
ax.tick_params(axis="y", labelsize=10)
ax.tick_params(axis="z", labelsize=10)

plt.show()

In [ ]:
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('time (ms)', fontsize=12)
#ax.set_ylabel('glucose [mM]', fontsize=18)
ax.set_ylabel('[ATP] (mM)', fontsize=12)
ax.set_zlabel('PSC change [%]', fontsize=12)

x = np.linspace(-100,100,200)
#y = np.linspace(8,30,30)  # Glucose
y = np.linspace(0.001,.1,30)  # ATP
X, Y = np.meshgrid(x, y)
zs = []
for A in y:
    zs.append(stimuli_60s_e_sngl(50, x, A, 'mu'))
zs = np.array(zs)
Z = zs.reshape(X.shape)

surf = ax.plot_surface(X, Y, Z, rstride=5, cstride=5, cmap='cool', alpha=.6,
                        linewidth=0, antialiased=True, vmin=np.nanmin(Z), vmax=np.nanmax(Z))

ax.plot_wireframe(X, Y, Z, rstride=5, cstride=5)
plt.title(r'STDP: 60 seg 1 Hz stimulation. dependence in $\mu:=\mu(A)$', fontsize=20)
ax.tick_params(axis="x", labelsize=10)
ax.tick_params(axis="y", labelsize=10)
ax.tick_params(axis="z", labelsize=10)

plt.show()

In [ ]:
A = np.linspace(0.001,.2,30)
intns = 2
x = intns*(A - G_h)/G_h
print(lam)
lm = .4
lam_n = lam + (lm/(1 + np.exp(-x)) - lm/2)
lam_n = lam_n*(lam_n>=0)
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.plot(A, lam_n)
ax.plot(A, np.ones(len(lam_n))*lam)
print(lam_n)

In [ ]:
A = np.linspace(0.001,.2,30)
intns = 3
x = intns*(A - G_h)/G_h
m = .2
print(mu)
mu_n = mu + (m/(1 + np.exp(-x)) - m/2)
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.plot(A, mu_n)
ax.plot(A, np.ones(len(mu_n))*mu)
print(lam_n)

In [ ]:
A = np.linspace(0.001,.2,30)
intns = 2.2
x = intns*(A - G_h)/G_h
a = -.2
print(alpha)
alpha_n = alpha + (a/(1 + np.exp(-x)) - a/2)
fig = plt.figure(figsize = (8,6))
ax = fig.add_subplot(111)
ax.plot(A, alpha_n)
ax.plot(A, np.ones(len(alpha_n))*alpha)
print(alpha_n)